In [ ]:
import biosteam as bst

In [ ]:
bst.main_flowsheet.set_flowsheet('BnAc production')
#Create a general chemicals to represent the components of fermentation

chemicals = bst.Chemicals([
    'Water', #Define common chemicals by name
   
    #Medium
    bst.Chemical('Citric acid', phase='s'), #Set the phase for chemicals not in vapor-liquid equilibrium
    bst.Chemical('TMS', rho = 1000, default = True, search_db=False, phase = 'l', MW = 18.),
    bst.Chemical('KH2PO4', phase = 's'),
    bst.Chemical('(NH4)2HPO4', phase = 's'),
    bst.Chemical('NaOH', phase='s'),
    bst.Chemical('Tyrosine', default = True, phase = 's'),
    
    #LB
    bst.Chemical('Yeast_extract', rho = 1540, default = True, search_db=False, phase = 's', MW = 1.),
    bst.Chemical('Tryptone', rho = 1540, default = True, search_db=False, phase = 's', MW = 1.),
    bst.Chemical('NaCl'),
    
    #Feed
    bst.Chemical('Glucose', phase='s'),
    bst.Chemical('MgSO4.7H2O', default = True, phase='s'),
    bst.Chemical('NH4OH', default = True, phase = 'l'),
    
    #Supplements
    bst.Chemical('Thiamine', default = True, phase='s'),
    bst.Chemical('IPTG', phase='s'),
    bst.Chemical('Tributyrin'),
    
    #Antibiotics
    bst.Chemical('Ampicillin', default = True, phase='s'),
    bst.Chemical('Kanamycin', phase='s'),
    bst.Chemical('Streptomycin', default = True, phase='s'),
    
    #Products
    bst.Chemical('Benzyl_acetate'),
    bst.Chemical('Ecoli_biomass', rho = 1540, default = True, search_db=False, phase = 's', MW = 1.),
    bst.Chemical('Spent_medium', default = True, search_db=False, phase = 'l'),
    
    #Raw materials for operating boilerturbogenerator
    bst.Chemical('CO2', phase = 'g'),
    bst.Chemical('O2', phase = 'g'),
    bst.Chemical('N2', phase = 'g'),
    bst.Chemical('P4O10',
                  rho=1540, # Density [kg/m3]
                  default=True,  # Default other chemicals properties like viscosity to that of water at 25 C
                  phase='s'),
    bst.Chemical('CaO',
                  Cp=1.02388,
                  rho=1540,
                  default=True,
                  search_db=False,
                  phase='s',
                  formula='CaO'),
    bst.Chemical('HCl'),
    bst.Chemical('SO2'),
    bst.Chemical('CaSO4'),
    bst.Chemical('Ash',
                  rho=1540,
                  Cp=0.37656,
                  default=True,
                  search_db=False,
                  phase='s',
                  MW=1.),
    bst.Chemical('CH4')
])
bst.settings.set_thermo(chemicals)



In [ ]:
bst.settings.CEPCI = 802.6 #2023 Jan	802.6
bst.settings.electricity_price = 0.100 #[USD/kWh] Arbitrary
#Steam is produced on-site by a boiler,
#so make it the only available heating agent.
steam_utility = bst.settings.get_agent('low_pressure_steam')
bst.settings.heating_agents = [steam_utility]
steam_utility.heat_transfer_efficiency = 0.9
steam_utility.T = 529.2
steam_utility.P = 44e5

#Steam, cooling water, and chilled water are regenerated by
#on-site facilities. The regeneration and heat transfer
#prices are given accounted for by the capital cost and
#electricity consumption of these facilities
steam_utility.regeneration_price = 0.
bst.settings.get_agent('cooling_water').regeneration_price = 0.
bst.settings.get_agent('chilled_water').heat_transfer_price = 0.

In [ ]:
#Raw material price (USD/kg)
#https://catcost.chemcatbio.org/materials-library
price = {
    'Water': 0.000353,
    
    #Medium
    'Citric acid': 1.87, #Citric acid, USP, anhydrous, granular, 50-lb. bgs., t.l. (2021)
    'TMS': 0.01, #consider as 1/200 diluted HCl: 0.14/200 USD/kg [Hydrochloric acid, 22 deg. Be, US Gulf dom. ex-works USG (2021)]
    'KH2PO4': 2.77,# repliace with NaH2PO4 price; Sodium phosphate, monobasic dms., c.l., t.l., works, frt. equald. (2021)
    '(NH4)2HPO4': 0.35,# Ammonium phosphate, (diammonium phosphate) fert. grade, min 18% N, 20% P, bulk, c.l., spot, f.o.b. Fla. works (2021)
    'NaOH': 0.86, #Caustic soda (sodium hydroxide), liq., spot export f.o.b. USG
    'Tyrosine': 32.3, #https://www.chemicalbook.com/ProductList_En.aspx?cbn=CB1269334&c=25KG&left=True#J_Condition
    
    #LB
    'Yeast_extract': 50.1,# Alfa Aesar; https://www.chemicalbook.com/ProductList_En.aspx?kwd=yeast%20extract&c=1kg&left=True#J_Condition
    'Tryptone': 81.2,#https://www.chemicalbook.com/ProductList_En.aspx?cbn=CB2156015&c=1kg&left=True#J_Condition
    'NaCl': 0.46,#https://www.chemicalbook.com/SupplierPriceList_EN.aspx?cbn=CB4104636&c=1000KG#price
    
    #Feed
    'Glucose': 0.58, #https://www.selinawamucii.com/insights/prices/united-states-of-america/glucose/
    'MgSO4.7H2O': 0.67,# Magnesium sulfate, 10% Mg., (epsom salts) tech. bgs., t.l., works
    'NH4OH': 6.4, #https://www.chemicalbook.com/ProductList_En.aspx?kwd=ammonium%20hydroxide&c=25kg&left=True#J_Condition
    
    #Supplements
    'Thiamine': 520, #https://www.sigmaaldrich.com/US/en/product/sial/t4625
    'IPTG': 1480, #https://www.chemicalbook.com/ProductList_En.aspx?cbn=CB2121723&c=250G&left=True#J_Condition
    'Tributyrin': 111, #https://www.chemicalbook.com/ProductList_En.aspx?kwd=tributyrin&c=1kg&left=True#J_Condition
    
    #Antibiotics
    'Ampicillin': 580, # ampicillinl sodium salt; https://www.chemicalbook.com/ProductList_En.aspx?cbn=CB8151850&c=250G&left=True#J_Condition
    'Kanamycin': 1020, # kanamycin sulfate; https://www.chemicalbook.com/ProductList_En.aspx?cbn=CB5722316&c=500g&left=True#J_Condition
    'Streptomycin': 380, # Streptomycin sulfate; https://www.chemicalbook.com/ProductList_En.aspx?kwd=streptomycin%20sulfate&c=500g&left=True#J_Condition
    
    #Products
    'Benzyl_acetate': 100, #current minimum price: 56; https://www.chemicalbook.com/ProductList_En.aspx?cbn=CB8345095&c=500g&left=True#J_Condition
    'Ecoli_biomass': 0,
#    'Waste': -0.33,
    'Waste_water': 0.00031, #negative sign should be added for price calculation; -0.00031, #$0.31/m^3; https://www.sciencedirect.com/science/article/pii/S2352728522000124
    
    #Fuels and facilities
    'Steam': 0.017,
    'Gasoline': 0.756, #2 USD/gal
    'Lime': 0.077,
    'H3PO4': 0, # Not significant
    'HCl': 0.205,
}


In [ ]:
from biosteam import units
import numpy as np

#bst.main_flowsheet.set_flowsheet('BnAc_production')

#We can create streams and set component splits faster by defining chemical groups
"""
For MR_base
"""
MR_base_composition = {#weight ratio
    'Water': 0.83123, #1.7 L MR base per 2 L of final fermentation volume
    'Citric acid': 0.0008,
    'TMS': 0.005,
    'KH2PO4': 0.00667,
    '(NH4)2HPO4': 0.004,
    'NaOH': 0.0022,
    'Tyrosine': 0.0001,
}
MR_base_composition_sum = 0.
for i in MR_base_composition: MR_base_composition_sum += MR_base_composition[i]
print('MR_base_composition_sum: ' + str(MR_base_composition_sum))

chemicals.define_group(
    name = 'MR_base',
    IDs=['Water', 'Citric acid', 'TMS', 'KH2PO4', '(NH4)2HPO4', 'NaOH', 'Tyrosine'],
    composition=[MR_base_composition['Water'], MR_base_composition['Citric acid'], MR_base_composition['TMS'], MR_base_composition['KH2PO4'], MR_base_composition['(NH4)2HPO4'], MR_base_composition['NaOH'], MR_base_composition['Tyrosine']],
    wt=True, #Composition is given as weight
)

"""
For MR_supplement
"""
MR_supplement_composition = {#weight ratio
    'Water': 744, # 400 g/L glucose: 1.149 g/mL
    'Glucose': 400, # 400 g/L -> 1/20 dil: 20 g/L
    'MgSO4.7H2O': 16, # 16 g/L -> 1/20 dil: 0.8 g/L
}
MR_supplement_composition_sum = 0.
for i in MR_supplement_composition: MR_supplement_composition_sum += MR_supplement_composition[i]
print('MR_supplement_composition_sum: ' + str(MR_supplement_composition_sum))

chemicals.define_group(
    name = 'MR_supplement',
    IDs = ['Water', 'Glucose', 'MgSO4.7H2O'],
    composition = [MR_supplement_composition['Water'], MR_supplement_composition['Glucose'], MR_supplement_composition['MgSO4.7H2O']],
    wt=True, #Composition is given as weight
)

"""
For MR_supplement2
"""

MR_supplement2_composition = {#weigt ratio
    'Water': 0.004, #1 mL 1000x ampicillin stock + 1 mL 1000x kanaycin stock + 1 mL 1000x streptomycin stock + 1 mL 1000x thiamine stock
    'Ampicillin': 0.0001, #Fianl conc.: 100 mg/L
    'Kanamycin': 0.00005,#Final conc.: 50 mg/L
    'Streptomycin': 0.00005,#Final conc.: 50 mg/L
    'Thiamine': 0.00001, #Final conc.: 10 mg/L
}
MR_supplement2_composition_sum = 0.
for i in MR_supplement2_composition: MR_supplement2_composition_sum += MR_supplement2_composition[i]
print('MR_supplement2_composition_sum: ' + str(MR_supplement2_composition_sum))

chemicals.define_group(
    name = 'MR_supplement2',
    IDs = ['Water', 'Ampicillin', 'Kanamycin', 'Streptomycin', 'Thiamine'],
    composition = [MR_supplement2_composition['Water'], MR_supplement2_composition['Ampicillin'], MR_supplement2_composition['Kanamycin'], MR_supplement2_composition['Streptomycin'], MR_supplement2_composition['Thiamine']],
    wt = True
)


"""
For Feed_base
"""
Feed_base_composition = {#weight ratio
    'Water': 582, # Feed density: 1.29 g/mL
    'Glucose': 700, # 700 g/L -> 1/20 dil: 20 g/L
    'MgSO4.7H2O': 8, # 8 g/L -> 1/20 dil: 0.8 g/L
}
Feed_base_composition_sum = 0.
for i in Feed_base_composition: Feed_base_composition_sum += Feed_base_composition[i]
print('Feed_base_composition_sum: ' + str(Feed_base_composition_sum))

chemicals.define_group(
    name = 'Feed_base',
    IDs = ['Water', 'Glucose', 'MgSO4.7H2O'],
    composition = [Feed_base_composition['Water'], Feed_base_composition['Glucose'], Feed_base_composition['MgSO4.7H2O']],
    wt=True, #Composition is given as weight
)

"""
For Feed_supplement
"""
Feed_supplement_composition = {#weight ratio
    'Water': 0.005, #1 mL 1000x ampicillin stock + 1 mL 1000x kanaycin stock + 1 mL 1000x streptomycin stock + 1 mL 1000x thiamine stock + 1 mL 1000x IPTG
    'Ampicillin': 0.0001, #Fianl conc.: 100 mg/L
    'Kanamycin': 0.00005,#Final conc.: 50 mg/L
    'Streptomycin': 0.00005,#Final conc.: 50 mg/L
    'Thiamine': 0.00001,#Final conc.: 10 mg/L
    'TMS': 0.005, #Final conc.: 5 mL/L
    'IPTG': 0.00023831, #Final conc.: 1 mM IPTG = 238.31 mg/L IPTG
}
Feed_supplement_composition_sum = 0.
for i in Feed_supplement_composition: Feed_supplement_composition_sum += Feed_supplement_composition[i]
print('Feed_supplement_composition_sum: ' + str(Feed_supplement_composition_sum))

chemicals.define_group(
    name = 'Feed_supplement',
    IDs = ['Water', 'Ampicillin', 'Kanamycin', 'Streptomycin', 'Thiamine', 'TMS', 'IPTG'],
    composition = [Feed_supplement_composition['Water'], Feed_supplement_composition['Ampicillin'], Feed_supplement_composition['Kanamycin'], Feed_supplement_composition['Streptomycin'], Feed_supplement_composition['Thiamine'], Feed_supplement_composition['TMS'], Feed_supplement_composition['IPTG']],
    wt = True
)

"""
For LB
"""
LB_composition = {#weight ratio
    'Water': 0.975,
    'Yeast_extract': 0.005,
    'Tryptone': 0.01,
    'NaCl': 0.01
}
LB_composition_sum = 0.
for i in LB_composition: LB_composition_sum += LB_composition[i]
print('LB_composition_sum: ' + str(LB_composition_sum))

chemicals.define_group(
    name = 'LB',
    IDs = ['Water', 'Yeast_extract', 'Tryptone', 'NaCl'],
    composition = [LB_composition['Water'], LB_composition['Yeast_extract'], LB_composition['Tryptone'], LB_composition['NaCl']],
    wt=True
)

                                      
lime = bst.Stream('lime',
                  CaO=333.00, Water=2200.00, units='kg/hr',
                  price=price['Lime'])  # to P5

In [ ]:
V_Fermentor = 40000 #[kg/hr]; 10,000 metric ton/yr & 200 working days/yr -> ~2 ton/hr ~2000 kg/hr
MR_base_components = bst.Stream('MR_base_components',
                         MR_base = V_Fermentor * 0.85 * 1.2, units='kg/hr',
                         price = (price['Water'] * MR_base_composition['Water']
                                 + price['Citric acid'] * MR_base_composition['Citric acid']
                                 + price['TMS'] * MR_base_composition['TMS']
                                 + price['KH2PO4'] * MR_base_composition['KH2PO4']
                                 + price['(NH4)2HPO4'] * MR_base_composition['(NH4)2HPO4']
                                 + price['NaOH'] * MR_base_composition['NaOH']
                                 + price['Tyrosine'] * MR_base_composition['Tyrosine']) / MR_base_composition_sum
)
MR_supplement_components = bst.Stream('MR_supplement_components',
                                      MR_supplement = (V_Fermentor * 1.2) * 0.05 * 1.16, units = 'kg/hr',
                                      price = (price['Water'] * MR_supplement_composition['Water']
                                                  + price['Glucose'] * MR_supplement_composition['Glucose']
                                                  + price['MgSO4.7H2O'] * MR_supplement_composition['MgSO4.7H2O']) / MR_supplement_composition_sum
)
MR_supplement2_components = bst.Stream('MR_supplement2_components',
                                       MR_supplement2 = (V_Fermentor * 1.2) * 0.00421, #1 mL 1000x ampicillin stock + 1 mL 1000x kanaycin stock + 1 mL 1000x streptomycin stock + 1 mL 1000x thiamine stock
                                       units = 'kg/hr',
                                       price = (price['Water'] * MR_supplement2_composition['Water']
                                                  + price['Ampicillin'] * MR_supplement2_composition['Ampicillin']
                                                  + price['Kanamycin'] * MR_supplement2_composition['Kanamycin']
                                                  + price['Streptomycin'] * MR_supplement2_composition['Streptomycin']
                                                  + price['Thiamine'] * MR_supplement2_composition['Thiamine']
                                                 ) / MR_supplement2_composition_sum
)
Feed_base_components = bst.Stream('Feed_base_components',
                                  Feed_base = V_Fermentor * 0.240 / 2.0, #240 g per 2 L starting volume
                                  units = 'kg/hr',
                                  price = (price['Water'] * Feed_base_composition['Water']
                                           + price['Glucose'] * Feed_base_composition['Glucose']
                                           + price['MgSO4.7H2O'] * Feed_base_composition['MgSO4.7H2O']) / Feed_base_composition_sum
)
Feed_supplement_components = bst.Stream('Feed_supplement_components',
                                         Feed_supplement = (V_Fermentor * 0.240 / 2.0) * 0.01044831, #1 mL + 1 mL + 1 mL + 1 mL + 1 mL + 5 mL per 1 L Feed_base (Feed_base: 240 g per 2 L starting volume)
                                         units = 'kg/hr',
                                         price = (price['Water'] * Feed_supplement_composition['Water']
                                                  + price['Ampicillin'] * Feed_supplement_composition['Ampicillin']
                                                  + price['Kanamycin'] * Feed_supplement_composition['Kanamycin']
                                                  + price['Streptomycin'] * Feed_supplement_composition['Streptomycin']
                                                  + price['Thiamine'] * Feed_supplement_composition['Thiamine']
                                                  + price['TMS'] * Feed_supplement_composition['TMS']
                                                  + price['IPTG'] * Feed_supplement_composition['IPTG']
                                                 ) / Feed_supplement_composition_sum
)
LB_components = bst.Stream('LB_components',
                           LB = V_Fermentor * 0.1, units = 'kg/hr',
                           price = (price['Water'] * LB_composition['Water']
                                    + price['Yeast_extract'] * LB_composition['Yeast_extract']
                                    + price['Tryptone'] * LB_composition['Tryptone']
                                    + price['NaCl'] * LB_composition['NaCl']) / LB_composition_sum
)


Tyr_NaOH = bst.Stream('Tyr_NaOH',
                      Water = V_Fermentor * 0.002, #4 mL per 2 L starting volume
                      Tyrosine = V_Fermentor * 0.002 * 0.05, #50 g/L Tyrosine
                      NaOH = V_Fermentor * 0.002 * 0.022, #0.55 M NaOH = 22 g/L NaOH
                      units = 'kg/hr',
                      price = (price['Water'] * 1
                               + price['Tyrosine'] * 0.05
                               + price['NaOH'] * 0.022)/(1 + 0.05 + 0.022)
)

Tributyrin_supplement = bst.Stream('Tributyrin_supplement',
                                   Tributyrin = V_Fermentor * 0.2 * 0.1 *1.05, #Assume 10% replacement; density = 1.05 g/mL
                                   units = 'kg/hr',
                                   price = price['Tributyrin']
)

IPTG_supplement = bst.Stream('IPTG_supplement',
                            Water = V_Fermentor * 1.2 * 0.001, #Initial volume + delayed inoculum (0.2x) -> 1/1000 dilution of 1000x IPTG
                            IPTG = V_Fermentor * 1.2 * 0.001 * 0.23831, #Final conc.: 1 mM IPTG = 238.31 mg/L IPTG, Initial volume + delayed inoculum (0.2x) -> 1/1000 dilution of 1000x IPTG
                            units = 'kg/hr',
                            price = (price['Water'] * 1
                               + price['IPTG'] * 0.23831)/(1 + 0.05 + 0.23831)
)

Base_solution = bst.Stream('Base_solution',
                           NH4OH = V_Fermentor * 0.03,#60 g per 2 L of initial fermentor volume
                           units = 'kg/hr',
                           price = price['NH4OH']
)

Seed1_seed = bst.Stream('Seed1_seed',
                        LB = (V_Fermentor * 0.1) * 0.02, #Inoculate 1/50 volume of seed culture; 1 mL in 50 mL lB
                        Ecoli_biomass = (V_Fermentor * 0.1 * 0.02) * 0.00033 * 2.5,#OD600 = 2.5; OD600 = 1 -> 0.33 g/L biomass
                        units = 'kg/hr',
                        price = ((price['Water'] * LB_composition['Water']
                                    + price['Yeast_extract'] * LB_composition['Yeast_extract']
                                    + price['Tryptone'] * LB_composition['Tryptone']
                                    + price['NaCl'] * LB_composition['NaCl']) / LB_composition_sum
                                    + price['Ecoli_biomass'] * 0.00033*2.5)/(1 + 0.00033*2.5)
)

Seed2_seed = bst.Stream('Seed2_seed',
                        LB = (V_Fermentor * 0.2) * 0.02, #Inoculate 1/50 volume of seed culture; 1 mL in 50 mL lB
                        Ecoli_biomass = (V_Fermentor * 0.2 * 0.02) * 0.00033 * 2.5,#OD600 = 2.5; OD600 = 1 -> 0.33 g/L biomass
                        units = 'kg/hr',
                        price = ((price['Water'] * LB_composition['Water']
                                    + price['Yeast_extract'] * LB_composition['Yeast_extract']
                                    + price['Tryptone'] * LB_composition['Tryptone']
                                    + price['NaCl'] * LB_composition['NaCl']) / LB_composition_sum
                                    + price['Ecoli_biomass'] * 0.00033*2.5)/(1 + 0.00033*2.5)
)


In [ ]:
"""
Unit operations
"""

Air_saturation_time = 16 #16 [hr]
Cleaning_time = 6 #6 [hr]
price['Benzyl_acetate'] = 100 #[USD/kg]; min: 56 USD/kg; max: 4800 USD/kg
BnAcTiter = 0.0021 #0.0021 [kg/L]
IRR = 0.15 #0.15
"""
Area 100: Sterilize MR_base_components
"""
#Mixing MR_base_components
M101 = units.Mixer('M101', MR_base_components)

#Heat up MR_base_components at 121C
H101 = units.HXutility('H101', M101-0, T=121 + 273.15)

#Autoclave MR_base_components at 121 C & 2 atm for 1 hr
T101 = units.StorageTank('T101', H101-0, tau = 1)

#Cool down the autoclaved MR_base_components to 30 C
H102 = units.HXutility('H102', T101-0, T = 30 + 273.15)


"""
Area 200: Sterilize MR_supplement_components
"""
#Mixing MR_supplement_components
M201 = units.Mixer('M201', MR_supplement_components)

#Heat up MR_supplement_components at 121C
H201 = units.HXutility('H201', M201-0, T=121 + 273.15)

#Autoclave MR_supplement_components at 121 C & 2 atm for 0.5 hr
T201 = units.StorageTank('T201', H201-0, tau = 1)

#Cool down the autoclaved MR_supplement_components to 30 C
H202 = units.HXutility('H202', T201-0, T = 30 + 273.15)


"""
Area 300: Mix and divide MR medium
"""
#Mix MR_base_components, MR_supplement_components, and MR_supplement2_compmonents
M301 = units.Mixer('M301', ins = [H102-0, H202-0, MR_supplement2_components])


#Split MR medium into StorageTank for delayed inoculation and BatchBioreactor for main fermentation
S301 = units.Splitter('S301', M301-0, split = 1/6)


"""
Area 400: Prepare delayed inoculum (Seed2)
"""
#Store and air-saturate MR medium for delayed inoculum
T401 = units.StorageTank('T401', S301-0, tau = Air_saturation_time)

#Culture delayed inoculum for 15 h at 30 C
B401 = units.BatchBioreactor('B401', ins = [T401-0, Seed2_seed], outs=('Vent2', bst.Stream('Seed2',
                                                                                           Ecoli_biomass = (T401.outs[0].F_mass + Seed2_seed.F_mass) * 0.00333, # OD600 = 10 -> 3.33 g/L Ecoli_biomass
                                                                                           Spent_medium = (T401.outs[0].F_mass + Seed2_seed.F_mass) * 0.99666,
                                                                                           units = 'kg/hr',
                                                                                           T = 30 + 273.15
                                                                                          )),
                              T = 30 + 273.15, tau = 20, V = 20 #[m^3] #V_Fermentor/100 * 0.5 #[m^3]; V_Fermentor in [L/hr]
                            )
B401.tau_0 = Cleaning_time #6 h; Cleaning and unlaoding time (hr)
@B401.add_specification(run = True)
def adjust_Seed2_flow():
    B401.outs[1] = bst.Stream('Seed2',
                              Ecoli_biomass = (T401.outs[0].F_mass + Seed2_seed.F_mass) * 0.00333, # OD600 = 10 -> 3.33 g/L Ecoli_biomass
                              Spent_medium = (T401.outs[0].F_mass + Seed2_seed.F_mass) * 0.99666,
                              units = 'kg/hr',
                              T = 30 + 273.15
                             )


"""
Area 500: Prepare inoculum for fermentation (Seed1)
"""
#Mix LB_components
M501 = units.Mixer('M501', LB_components)

#Heat up LB_components
H501 = units.HXutility('H501', M501-0, T= 121 + 273.15)

#Autoclave LB_components at 121 C & 2 atm for 1 hr
T501 = units.StorageTank('T501', H501-0, tau = 1)

#Cool down the autoclaved LB_components to 30 C
H502 = units.HXutility('H502', T501-0, T = 37 + 273.15)

#Air-saturate LB medium
T502 = units.StorageTank('T502', H502-0, tau = Air_saturation_time) # + 3 - 2 - 12)

#Culture inoculum for 12 h at 37 C
B501 = units.BatchBioreactor('B501', ins = [T502-0, Seed1_seed], outs=('Vent0', bst.Stream('Seed1',
                                                                                           Ecoli_biomass = (T502.outs[0].F_mass + Seed1_seed.F_mass) * 0.00333, # OD600 = 10 -> 3.33 g/L Ecoli_biomass
                                                                                           Spent_medium = (T502.outs[0].F_mass + Seed1_seed.F_mass) * 0.99666,
                                                                                           units = 'kg/hr',
                                                                                           T = 30 + 273.15
                                                                                          )),
                              T = 37 + 273.15, tau = 12, V = 20 #[m^3] #V_Fermentor/100 * 0.2 #[m^3]; V_Fermentor in [L/hr]
                            )
B501.tau_0 = Cleaning_time #6 hr; Cleaning and unlaoding time (hr)
@B501.add_specification(run = True)
def adjust_Seed1_flow():
    B501.outs[1].imass['Ecoli_biomass', 'Spent_medium'] = [(T502.outs[0].F_mass + Seed1_seed.F_mass) * 0.00333, # OD600 = 10 -> 3.33 g/L Ecoli_biomass
                                                           (T502.outs[0].F_mass + Seed1_seed.F_mass) * 0.99666
                                                          ]


"""
Area 600: Prepare feed
"""
#Mix Feed_base_components
M601 = units.Mixer('M601', Feed_base_components)

#Heat up Feed_base_components at 121C
H601 = units.HXutility('H601', M601-0, T=121 + 273.15)

#Autoclave Feed_base_components at 121 C & 2 atm for 1 hr
T601 = units.StorageTank('T601', H601-0, tau = 1)

#Cool down the autoclaved Feed_base_components to 30 C
H602 = units.HXutility('H602', T601-0, T = 30 + 273.15)

#Mix Feed_base_components and Feed_supplement_components
M602 = units.Mixer('M602', ins = [H602-0, Feed_supplement_components])


"""
Area 700: Run fermentation
"""
#Air-saturate MR meidum
T701 = units.StorageTank('T701', S301-1, tau = Air_saturation_time)

#Since BatchBioreactor accepts up to 2 inlets, Mix air-saturated MR medium and Seed1
M701 = units.Mixer('M701', ins = [T701-0, B501-1])

#Mix fresh and recycled tributyrin
recycled_tributyrin = bst.Stream('recycled_tributyrin', Tributyrin = 1.)
fresh_tributyrin = bst.Stream('fresh_tributyrin', Tributyrin = 1.)
M702 = units.Mixer('M702', ins = [fresh_tributyrin, recycled_tributyrin], outs = [Tributyrin_supplement])
@M702.add_specification(run=True)
def adjust_fresh_tributyrin():
    M702.ins[0].imass['Tributyrin'] = V_Fermentor * 0.2 *1.05 - recycled_tributyrin.imass['Tributyrin'] #Assume 10% replacement; density = 1.05 g/mL
    M702.ins[0].price = M702.ins[0].F_mass * price['Tributyrin']

#Mix supplements
M703 = units.Mixer('M703', ins = [B401-1, M602-0, Tyr_NaOH, M702-0, IPTG_supplement, Base_solution])
@M703.add_specification(run = True)
def adjust_Seed2_flow():
    M703.ins[0] = B401.outs[1]

#Main fermentation
B701 = units.BatchBioreactor('B701', ins = [M701-0, M703-0], outs=('Vent1', bst.Stream('Fermentation_broth',
                                                                                       Tributyrin = V_Fermentor * 0.2 * 1.05,
                                                                                       Ecoli_biomass = V_Fermentor * 0.005,
                                                                                       Spent_medium = (V_Fermentor + Tyr_NaOH.F_mass + IPTG_supplement.F_mass + Base_solution.F_mass) * (0.99 - BnAcTiter),
                                                                                       Benzyl_acetate = (V_Fermentor + Tyr_NaOH.F_mass + IPTG_supplement.F_mass + Base_solution.F_mass) * BnAcTiter, #0.023, #0.0021,
                                                                                       units = 'kg/hr',
                                                                                       T = 30 + 273.15
                                                                                      )),
                              T = 30 + 273.15, tau = 96, V = 200 #[m^3] #V_Fermentor/100 * 15 #[m^3]; V_Fermentor in [L/hr]
                            )
B701.tau_0 = Cleaning_time #6 hr; Cleaning and unlaoding time (hr)



#Separate aqueous and organic phases of fermentation broth
C801 = units.SolidLiquidsSplitCentrifuge('C801', B701-1, outs = ['aqueous_phase', 'organic_phase_1', 'biomass_slurry'],
                                         solids_split = {'Ecoli_biomass': 1.0, 'Tributyrin': 0.1, 'Benzyl_acetate': 0.1},
                                         aqueous_split = {'Tributyrin': 1.0, 'Benzyl_acetate': 1.0, 'Ecoli_biomass': 1.0}, #heavier phase; organic_phase1 is heavier than aqueous phase in this case
                                         moisture_content=0.0
                                        )
@C801.add_specification(run=True)
def adjust_waste_cost():
    C801.outs[0].price = C801.outs[0].F_mass * price['Waste_water'] * (-1.0)
    
#Filter out biomass
F801 = units.RotaryVacuumFilter('F801', C801-2, outs = ['biomass', 'organic_phase_2'],
                                split = dict(Tributyrin = 0.,
                                             Benzyl_acetate = 0.,
                                             Ecoli_biomass = 1.0
                                            ))
M801 = units.Mixer('M801', ins = [C801-1, F801-1], outs=['organic_phase'])
    
"""
Area 900: Isolate product, recycle tributyrin, and process wastes
"""

""" Distillation tower """
#Pump organic phase from the Mixer M801 to Distillation column
DistP = 101325 * 0.1
P901 = units.Pump('P901', M801-0, P = DistP)


#Heat up before distillation column
#Exchange heat with stillage
bottoms_product = bst.Stream() # Bottoms product from distillation column; connect later
H901 = units.HXprocess('H901', ins = [P901-0, bottoms_product], outs = ['', 1-M702],
                        U=1.28)

#Distilation column
D901 = units.BinaryDistillation('D901',
                                 H901-0, P = DistP,
                                 Lr = 0.999, Hr = 0.999999, #Light and heavy key recoveries
                                 LHK = ('Benzyl_acetate', 'Tributyrin'),
                                 k = 1.25, #Ratio of actual reflux over minimum reflux
                                 Rmin = 0.01, #Minimum allowable reflux ratio
)
D901.boiler.U = 1.85

#Transport bottoms product to heat exchanger
P902 = units.Pump('P902', D901-1, bottoms_product)

#Cool down top distillate
H902 = units.HXutility('H902', D901-0, V=0, T = 25 + 273.15)
#BnAc = bst.Stream('benzyl_acetate', price = price['Benzyl_acetate'])
T901 = bst.StorageTank('T901', ins = H902-0, outs = ['benzyl_acetate']) #[BnAc])
@T901.add_specification(run = True)
def adjust_Product_stream_price():
    T901.outs[0].price = T901.outs[0].F_mass * price['Benzyl_acetate']
  

#Burn bioimass to generate energy
s = bst.main_flowsheet.stream
BT = bst.BoilerTurbogenerator('BT',
                              (F801-0, '', 'boiler_makeup_water', 'natural_gas', '', ''),
                              boiler_efficiency = 0.80,
                              turbogenerator_efficiency=0.85
                             )

CT = bst.CoolingTower('CT')
makeup_water = bst.Stream('makeup_water', price = 0.000254)

CWP = bst.ChilledWaterPackage('CWP')
PWC = bst.ProcessWaterCenter('PWC',
                             ins = (bst.Stream(), makeup_water, bst.Stream()),
                             outs=(),
                             makeup_water_streams = (s.cooling_tower_makeup_water,
                                                     s.boiler_makeup_water),
                             process_water_streams = (s.cooling_tower_makeup_water,
                                                     s.boiler_makeup_water)
                            )
HXN = bst.HeatExchangerNetwork('HXN', units = [D901.boiler, D901.condenser]
                              )

bst.main_flowsheet.diagram(format = 'svg')

In [ ]:
BnAc_production_sys = bst.main_flowsheet.create_system('BnAc_production_sys')
BnAc_production_sys.simulate()
bst.main_flowsheet.diagram(format = 'svg')

In [ ]:
from matplotlib import pyplot as plt
D901.plot_stages()
plt.show()

In [ ]:
UnitGroup = bst.process_tools.UnitGroup
Medium_preparation = UnitGroup('Medium preparation',
                               [M101, H101, T101, H102,
                                M201, H201, T201, H202,
                                M301
                               ])
Seed1_preparation = UnitGroup('Seed1 preparation',
                              [M501, H501, T501, H502, T502, B501
                              ])
Seed2_preparation = UnitGroup('Seed2 preparation',
                              [S301,
                               T401, B401
                              ])
Feed_preparation = UnitGroup('Feed preparation',
                             [M601, H601, T601, H602, M602
                             ])
BnAc_production = UnitGroup('BnAc production',
                            [T701, M701,
                             B701,
                             M702, M703
                            ])
Phase_separation = UnitGroup('Phase separation',
                              [C801, F801, M801
                              ])
BnAc_purification = UnitGroup('BnAc purification',
                              [P901, H901,
                               D901, P902, H902, H902, T901
                              ])
facilities = UnitGroup('Facilities', BnAc_production_sys.facilities)
groups = [Medium_preparation, Seed1_preparation, Seed2_preparation, Feed_preparation,
          BnAc_production, Phase_separation, BnAc_purification, facilities]
UnitGroup.df_from_groups(groups)

In [ ]:
bst.plots.plot_unit_groups(groups, fraction=True)

In [ ]:
class BnAc_TEA(bst.TEA):
    """
    Create a BnAc_TEA object for techno-economic analysis of a biorefinery [1]_.

    Parameters
    ----------
    system : System
        Should contain feed and product streams.
    IRR : float
        Internal rate of return (fraction).
    duration : tuple[int, int]
        Start and end year of venture (e.g. (2018, 2038)).
    depreciation : str
        'MACRS' + number of years (e.g. 'MACRS7').
    operating_days : float
        Number of operating days per year.
    income_tax : float
        Combined federal and state income tax rate (fraction).
    lang_factor : float
        Lang factor for getting fixed capital investment from
        total purchase cost. If no lang factor, estimate capital investment
        using bare module factors.
    startup_schedule : tuple[float]
        Startup investment fractions per year
        (e.g. (0.5, 0.5) for 50% capital investment in the first year and 50%
        investment in the second).
    WC_over_FCI : float
        Working capital as a fraction of fixed capital investment.
    labor_cost : float
        Total labor cost (USD/yr).
    fringe_benefits : float
        Cost of fringe benefits as a fraction of labor cost.
    property_tax : float
        Fee as a fraction of fixed capital investment.
    property_insurance : float
        Fee as a fraction of fixed capital investment.
    supplies : float
        Yearly fee as a fraction of labor cost.
    maintenance : float
        Yearly fee as a fraction of fixed capital investment.
    administration : float
        Yearly fee as a fraction of fixed capital investment.

    References
    ----------
    .. [1] Huang, H., Long, S., & Singh, V. (2016). Techno-economic analysis of biodiesel
        and ethanol co-production from lipid-producing sugarcane. Biofuels, Bioproducts
        and Biorefining, 10(3), 299–315. https://doi.org/10.1002/bbb.1640

    """

    def __init__(self, system, IRR, duration, depreciation, income_tax,
                 operating_days, lang_factor, construction_schedule, WC_over_FCI,
                 labor_cost, fringe_benefits, property_tax,
                 property_insurance, supplies, maintenance, administration):
        # Huang et. al. does not take into account financing or startup
        # so these parameters are 0 by default
        super().__init__(system, IRR, duration, depreciation, income_tax,
                         operating_days, lang_factor, construction_schedule,
                         startup_months=0, startup_FOCfrac=0, startup_VOCfrac=0,
                         startup_salesfrac=0, finance_interest=0, finance_years=0,
                         finance_fraction=0, WC_over_FCI=WC_over_FCI)
        self.labor_cost = labor_cost
        self.fringe_benefits = fringe_benefits
        self.property_tax = property_tax
        self.property_insurance = property_insurance
        self.supplies= supplies
        self.maintenance = maintenance
        self.administration = administration

    # The abstract _DPI method should take installed equipment cost
    # and return the direct permanent investment. Huang et. al. assume
    # these values are equal
    def _DPI(self, installed_equipment_cost):
        return installed_equipment_cost

    # The abstract _TDC method should take direct permanent investment
    # and return the total depreciable capital. Huang et. al. assume
    # these values are equal
    def _TDC(self, DPI):
        return DPI

    # The abstract _FCI method should take total depreciable capital
    # and return the fixed capital investment. Again, Huang et. al.
    # assume these values are equal.
    def _FCI(self, TDC):
        return TDC

    # The abstract _FOC method should take fixed capital investment
    # and return the fixed operating cost.
    def _FOC(self, FCI):
        return (FCI*(self.property_tax + self.property_insurance
                     + self.maintenance + self.administration)
                + self.labor_cost*(1+self.fringe_benefits+self.supplies))

In [ ]:
tea = BnAc_TEA(system = BnAc_production_sys, #system : System; Should contain feed and product streams.
               IRR = IRR, # float; Internal rate of return (fraction).
               duration = (2026, 2046), # tuple[int, int]; Start and end year of venture (e.g. (2018, 2038)).
               depreciation = 'MACRS7', # str; 'MACRS' + number of years (e.g. 'MACRS7').
               operating_days = 200, # float; Number of operating days per year.
               income_tax = 0.21, # float; Combined federal and state income tax rate (fraction).
               lang_factor = 3, # float; Lang factor for getting fixed capital investment from total purchase cost.
                                #If no lang factor, estimate capital investment using bare module factors.
               construction_schedule = (0.4, 0.6), # startup_schedule : tuple[float];
                                                   # Startup investment fractions per year
                                                   # (e.g. (0.5, 0.5) for 50% capital investment in the first year and 50% investment in the second).
               WC_over_FCI = 0.5, # float; Working capital as a fraction of fixed capital investment.
               labor_cost = 2.5e6, # float; Total labor cost (USD/yr).
               fringe_benefits = 0.4, # float; Cost of fringe benefits as a fraction of labor cost.
               property_tax = 0.001, # float; Fee as a fraction of fixed capital investment.
               property_insurance = 0.005, # float; Fee as a fraction of fixed capital investment.
               supplies = 0.20, # float; Yearly fee as a fraction of labor cost.
               maintenance = 0.01, # float; Yearly fee as a fraction of fixed capital investment.
               administration = 0.005 # float; Yearly fee as a fraction of fixed capital investment.
              )

tea.show() # Print TEA summary at current options
#Ignore the warings, these are taken care of internally

In [ ]:
print(tea.get_cashflow_table())

In [ ]:
#Find production cost:
products = [bst.main_flowsheet('benzyl_acetate')]
costs = tea.production_costs(products) #USD/yr
np.round(costs / 1e6) # million USD/yr

In [ ]:
#Solve the price of a stream at the break even poinnt:
raw_materials = [bst.main_flowsheet('MR_base_components'),
                 bst.main_flowsheet('MR_supplement_components'),
                 bst.main_flowsheet('MR_supplement2_components'),
                 bst.main_flowsheet('LB_components'),
                 bst.main_flowsheet('Feed_base_components'),
                 bst.main_flowsheet('Feed_supplement_components'),
                 bst.main_flowsheet('IPTG_supplement'),
                 bst.main_flowsheet('fresh_tributyrin'),
                 bst.main_flowsheet('Tyr_NaOH'),
                 bst.main_flowsheet('Base_solution')]
raw_materials_price = tea.solve_price(raw_materials) # USD/kg
round(raw_materials_price, 5)

In [ ]:
#Solve for the IRR at the break even point:
tea.IRR = tea.solve_IRR()
tea.show()

In [ ]:
#Save stream tables, utility requirements, itemized costs, TEA results, and a cash flow table:
tea.system.save_report('TEA_System_report.xlsx')
# Ignore the warning. The flowsheet is saved on the excel file
# as a really big image and Python thinks it could be a
# malicious file cause its so big.

In [ ]:
"""
Create parameter distributions
"""
#A triangular distribution is typically used when the parameter is uncertain within given limits,
#but is heuristically known to take a particular value.
#Create a triangular distribution:

from warnings import filterwarnings; filterwarnings('ignore')
from chaospy import distributions as shape
lower_bound = 0
most_probable = 0.5
upper_bound = 1
triang = shape.Triangle(lower_bound, most_probable, upper_bound)
print(triang)


In [ ]:
#A uniform distribution is used when the theoretical limits of the parameter is known,
#but no information is available to discern which values are more probable.
#Create a uniform distribution:

lower_bound1 = 0
upper_bound1 = 1
unif = shape.Uniform(lower_bound1, upper_bound1)
print(unif)

#A large set of distributions are available through chaospy,
#but generally triangular and uniform distributions are the most widely used
#to describe the uncertainty of parameters in Monte Carlo analyses.

In [ ]:
"""
Parameter objects
"""
#Create a model object:
"Model objects are used to evaluate metrics around multiple parameters of a system."

total_utility_cost = lambda: tea.utility_cost/10**6 #In 10^6 USD/yr
metrics = (bst.Metric('Internal rate of return', tea.solve_IRR, '%'),
           bst.Metric('Utility cost', total_utility_cost, '10^6 USD/yr')
          )
model = bst.Model(BnAc_production_sys, metrics)
model

In [ ]:
#Add cost parameters as "isolated" parameters
"An isolated parameter does not affect Unit objects in any way."


#Add water price as a 'cost' parameter
@model.parameter(name = 'Water price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(0.0002, 0.000353, 0.0005)
                )
def set_water_price(water_price):
    price['Water'] = water_price #[USD/kg] Arbitrary

#Add TMS price as a 'cost' parameter
@model.parameter(name = 'TMS price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(0.005, 0.01, 0.02)
                )
def set_TMS_price(TMS_price):
    price['TMS'] = TMS_price #[USD/kg] Arbitrary

#Add KH2PO4 price as a 'cost' parameter
@model.parameter(name = 'KH2PO4 price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(1.4, 2.77, 5.5)
                )
def set_KH2PO4_price(KH2PO4_price):
    price['KH2PO4'] = KH2PO4_price #[USD/kg] Arbitrary

#Add (NH4)2HPO4 price as a 'cost' parameter
@model.parameter(name = '(NH4)2HPO4 price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(0.17, 0.35, 0.7)
                )
def set_NH4_2HPO4_price(NH4_2HPO4_price):
    price['(NH4)2HPO4'] = NH4_2HPO4_price #[USD/kg] Arbitrary

#Add NaOH price as a 'cost' parameter
@model.parameter(name = 'NaOH price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(0.4, 0.86, 1.7)
                )
def set_NaOH_price(NaOH_price):
    price['NaOH'] = NaOH_price #[USD/kg] Arbitrary

#Add tyrosine price as a “isolated” parameter:
@model.parameter(name = 'Tyrosine price', kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(16, 32.3, 64)
                )
def set_tyrosine_price(tyrosine_price):
    price['Tyrosine'] = tyrosine_price

#Add yeast extract price as a 'cost' parameter
@model.parameter(name = 'Yeast extract price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(25, 50.1, 100)
                )
def set_yeast_extract_price(yeast_extract_price):
    price['Yeast_extract'] = yeast_extract_price #[USD/kg] Arbitrary

#Add tryptone price as a 'cost' parameter
@model.parameter(name = 'Tryptone price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(40, 81.2, 162)
                )
def set_tryptone_price(tryptone_price):
    price['Tryptone'] = tryptone_price #[USD/kg] Arbitrary

#Add NaCl price as a 'cost' parameter
@model.parameter(name = 'NaCl price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(0.2, 0.46, 0.9)
                )
def set_NaCl_price(NaCl_price):
    price['NaCl'] = NaCl_price #[USD/kg] Arbitrary

#Add glucose price as a “isolated” parameter:
@model.parameter(name = 'Glucose price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(0.29, 0.58, 1.1)
                )
def set_glucose_price(glucose_price):
    price['Glucoe'] = glucose_price #[USD/kg]
    
#Add MgSO4.7H2O price as a 'cost' parameter
@model.parameter(name = 'MgSO4.7H2O price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(0.33, 0.67, 1.3)
                )
def set_MgSO4_price(MgSO4_price):
    price['MgSO4.7H2O'] = MgSO4_price #[USD/kg] Arbitrary

#Add NH4OH price as a 'cost' parameter
@model.parameter(name = 'NH4OH price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(3.2, 6.4, 12.8)
                )
def set_NH4OH_price(NH4OH_price):
    price['NH4OH'] = NH4OH_price #[USD/kg] Arbitrary

#Add Thiamine price as a 'cost' parameter
@model.parameter(name = 'Thiamine price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(250, 520, 1000)
                )
def set_thiamine_price(thiamine_price):
    price['Thiamine'] = thiamine_price #[USD/kg] Arbitrary

#Add IPTG price as a 'cost' parameter
@model.parameter(name = 'IPTG price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(700, 1480, 3000)
                )
def set_IPTG_price(IPTG_price):
    price['IPTG'] = IPTG_price #[USD/kg] Arbitrary

#Add tributyrin price as a “isolated” parameter:
@model.parameter(name = 'Tributyrin price', kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(55, 111, 222)
                )
def set_tributyrin_price(tributyrin_price):
    price['Tributyrin'] = tributyrin_price

#Add ampicillin price as a 'cost' parameter
@model.parameter(name = 'Ampicillin price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(290, 580, 1160)
                )
def set_ampicillin_price(ampicillin_price):
    price['Ampicillin'] = ampicillin_price #[USD/kg] Arbitrary

#Add kanamycin price as a 'cost' parameter
@model.parameter(name = 'Kanamycin price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(560, 1020, 2040)
                )
def set_kanamycin_price(kanamycin_price):
    price['Kanamycin'] = kanamycin_price #[USD/kg] Arbitrary

#Add streptomycin price as a 'cost' parameter
@model.parameter(name = 'Streptomycin price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(190, 380, 760)
                )
def set_streptomycin_price(streptomycin_price):
    price['Streptomycin'] = streptomycin_price #[USD/kg] Arbitrary

#Add steam price as a 'cost' parameter
@model.parameter(name = 'Steam price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(0.008, 0.017, 0.034)
                )
def set_steam_price(steam_price):
    price['Steam'] = steam_price #[USD/kg] Arbitrary

#Add gasoline price as a 'cost' parameter
@model.parameter(name = 'Gasoline price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(0.37, 0.756, 1.5)
                )
def set_gasoline_price(gasoline_price):
    price['Gasoline'] = gasoline_price #[USD/kg] Arbitrary

#Add lime price as a 'cost' parameter
@model.parameter(name = 'Lime price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(0.04, 0.077, 0.15)
                )
def set_lime_price(lime_price):
    price['Lime'] = lime_price #[USD/kg] Arbitrary

#Add HCl price as a 'cost' parameter
@model.parameter(name = 'HCl price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(0.10, 0.205, 0.41)
                )
def set_HCl_price(HCl_price):
    price['HCl'] = HCl_price #[USD/kg] Arbitrary

#Add benzyl acetate price as a 'cost' parameter
@model.parameter(name = 'Benzyl acetate price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(76, 100, 150)
                )
def set_benzyl_acetate_price(benzyl_acetate_price):
    price['Benzyl_acetate'] = benzyl_acetate_price #[USD/kg] Arbitrary

#Add waste water price as a 'cost' parameter
@model.parameter(name = 'Waste water price',
                 kind = 'isolated', units = 'USD/kg',
                 distribution = shape.Triangle(0.00015, 0.00031, 0.0006) #(0.0006, -0.00031, -0.00015)
                )
def set_waste_water_price(waste_water_price):
    price['Waste_water'] = waste_water_price #[USD/kg] Arbitrary


#Add electicity price as a 'cost' parameter
@model.parameter(name = 'Electricity price',
                 kind = 'isolated', units = 'USD/kWh',
                 distribution = shape.Triangle(0.05, 0.100, 0.200)
                )
def set_electricity_price(electricity_price):
    bst.settings.electricity_price = electricity_price #[USD/kWh] Arbitrary

#Add income tax as a 'cost' parameter
@model.parameter(name = 'Income tax',
                 kind = 'isolated',
                 distribution = shape.Triangle(0.1, 0.21, 0.4)
                )
def set_income_tax(income_tax):
    tea.income_tax = income_tax


In [ ]:
#Add "design" parameters
"A design parameter only affects unit operation design"

@model.parameter(name = 'Fermentation time', element = B701, 
                 kind = 'design', units = 'hr',
                 distribution = shape.Triangle(84, 96, 108)
                )
def set_fermentation_time(fermentation_time):
    B701.tau = fermentation_time

In [ ]:
#Add coupled parameters
"A coupled parameter affects mass and energy balances of the system."

#Add BnAc titer as a “coupled” parameter:
@model.parameter(
    name = 'Benzyl acetate titer',
    element = B701, kind = 'coupled', units = 'g/L',
    distribution = shape.Triangle(1.9, 2.1, 2.3), #1-10 g/L BnAc
#    hook = lambda N: int(round(titer)) # Make sure value is an integer
)
def set_BnAc_titer(titer):
    B701.outs[1].imass['Benzyl_acetate'] = (V_Fermentor + Tyr_NaOH.F_mass + IPTG_supplement.F_mass + Base_solution.F_mass) * titer * 0.001

#Add recovered BnAc ratio as a “coupled” parameter:
@model.parameter(
    name = 'Benzyl acetate recovery ratio',
    element = D901, kind = 'coupled',
    distribution = shape.Triangle(0.9, 0.999, 0.999999), #90-99.9999% recovery of BnAc at the distillate
)
def set_BnAc_recovery_ratio(benzyl_acetate_recovery_ratio):
    D901.Lr = benzyl_acetate_recovery_ratio

#Add recovered tributyrin ratio as a “coupled” parameter:
@model.parameter(
    name = 'Tributyrin recovery ratio',
    element = D901, kind = 'coupled',
    distribution = shape.Triangle(0.999, 0.999999, 0.99999999), #90-99.999999% recovery of BnAc at the distillate
)
def set_BnAc_recovery_ratio(tributyrin_recovery_ratio):
    D901.Hr = tributyrin_recovery_ratio


#Add distillation pressure as a “coupled” parameter:
@model.parameter(
    name = 'Distillation pressure',
    element = D901, kind = 'coupled', units = 'Pa',
    distribution = shape.Triangle(101325 * 0.05, 101325 * 0.1, 101325 * 0.2),
)
def set_distillation_pressure(distillation_pressure):
    D901.P = distillation_pressure


#The decorator uses the function to create a Parameter object and add it to the model:
parameters = model.get_parameters()
parameters

In [ ]:
#Calling a Parameter object will update the parameter and results:
set_BnAc_titer_parameter = parameters[4]
set_BnAc_titer_parameter(2.1)
print('B701.outs[1] at 5 g/L BnAc:')
B701.outs[1].show(flow = 'kg/hr')

set_BnAc_titer_parameter(5)
print('B701.outs[1] at 5 g/L BnAc:')
B701.outs[1].show(flow = 'kg/hr')

set_BnAc_titer_parameter(10)
print('B701.outs[1] at 10 g/L BnAc:')
B701.outs[1].show(flow = 'kg/hr')

In [ ]:
"""
Evaluate metric given a sample
"""
#The model can be called to evaluate a sample of parameters.

model

In [ ]:
df_dct = model.get_distribution_summary()
df_dct['Triangle']

In [ ]:
model([0.6, 560, 10]) #Returns metrics (IRR and utility cost)

In [ ]:
"""
Monte Carlo
"""
N_samples = 500
rule = 'L' #For Latin-Hypercube sampling
np.random.seed(1234) #For consistent results
samples = model.sample(N_samples, rule)
#model.load_samples(samples)
#model.evaluate(
#    notify = 50 #Also print elapsed time after 50 simulations
#)
#
#model.table #All evaluations are stored as a pandas DataFrame

"""
Although the system uses the last solution as an initial guess for the next,
each scenario may be vastly different and there is no guarantee that this initial guess would be any better.
Luckily, BioSTEAM can minimize perturbations to the system between simulations by pre-sorting the scenarios:
"""
model.load_samples(
    samples,
    optimize = True # Optimize simulation order
)
model.evaluate(
    notify = 50 #Also print elapsed time after 50 simulations
)

model.table #All evaluations are stored as a pandas DataFrame

In [ ]:
"""
Sensitivity with Spearman’s rank order correlation
"""
#Model objects also presents methods for sensitivity analysis such as Spearman’s correlation,
#a measure of monotonicity between variables:

df_rho, df_p = model.spearman_r()
print(df_rho['Biorefinery', 'Internal rate of return [%]'])

In [ ]:
#Create a tornado plot of Spearman’s correlation between all parameters and IRR:
bst.plots.plot_spearman_1d(df_rho['Biorefinery', 'Internal rate of return [%]'],
                     index = [i.describe() for i in model.parameters],
                     name = 'IRR') #[%]')

In [ ]:
"""
Single point sensitivity
"""
#A quick way to evaluate sentivity is through single point sensitivity analysis,
#whereby a metric is evaluated at the baseline and at the lower and upper limits of each parameter.
#This method ignores the interactions between parameters and their distributions,
#but can help screen whether a system is sensitive to a given parameter.
#Model objects also facilitate this analysis:

baseline, lower, upper = model.single_point_sensitivity()
print('BASELINE')
print('--------')
print(baseline)
print()
print('LOWER')
print('-----')
print(lower)
print()
print('UPPER')
print('-----')
print(upper)

In [ ]:
#Create a tornado plot of the lower and upper values of the IRR:
IRR, utility_cost = model.metrics
metric_index = IRR.index
index = [i.describe(distribution=False) # Instead of displaying distribution, it displays lower, baseline, and upper values
         for i in model.parameters]
bst.plots.plot_single_point_sensitivity(100 * baseline[metric_index],
                                        100 * lower[metric_index],
                                        100 * upper[metric_index],
                                        name='IRR [%]',
                                        index=index)
#Note that blue represents the upper limit while red the lower limit.